In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'loss'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 10000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
INPUT = Path("../input/tabular-playground-series-aug-2021")

train = pd.read_csv(INPUT / "train.csv")
test = pd.read_csv(INPUT / "test.csv")
submission = pd.read_csv(INPUT / "sample_submission.csv")

# Preprocessing

In [5]:
scale_features = [col for col in test.columns if 'f' in col]

ss = StandardScaler()
train[scale_features] = ss.fit_transform(train[scale_features])
test[scale_features] = ss.transform(test[scale_features])

In [6]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [7]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [8]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=train[scale_features], y=train[target]):
  """
  """
  param_space = {
                'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS']),
            'loss_function': 'RMSE',
            'eval_metric': 'RMSE',
          #'task_type': 'GPU',
          'max_depth':trial.suggest_int('max_depth', 6, 8),
           'learning_rate':trial.suggest_uniform('learning_rate', 7.89e-3, 8.89e-3),
          'n_estimators':N_ESTIMATORS,
        'max_bin':trial.suggest_int('max_bin', 463, 513),
             'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 96, 106),
         'reg_lambda':trial.suggest_uniform('reg_lambda', 0.2329, 0.3329),
            'subsample': trial.suggest_uniform('subsample', 0.6536, 0.7036)
                }
            

  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred = np.zeros(y_va.shape[0])
  
  
      
  for inseed in seed_list:

        param_space['random_seed'] = inseed

        model =  ctb.CatBoostRegressor(**param_space)
        model.fit(
            X_tr, 
            y_tr,
            eval_set=[(X_va, y_va)],
            use_best_model=True,
            early_stopping_rounds=EARLY_STOPPING_ROUNDS,
            verbose=VERBOSE
        )


        pred += model.predict(X_va) / len(seed_list)
        #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  rmse = mean_squared_error(y_va, pred, squared=False)
      
  
  return rmse

In [9]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-26 23:41:58,885] A new study created in memory with name: no-name-a6d105e8-c5cc-4732-9378-4e25aad16ce1


0:	learn: 7.9595147	test: 7.8603973	best: 7.8603973 (0)	total: 225ms	remaining: 37m 31s


1000:	learn: 7.8091510	test: 7.7976945	best: 7.7976945 (1000)	total: 2m 20s	remaining: 20m 58s


2000:	learn: 7.7010885	test: 7.7814423	best: 7.7814423 (2000)	total: 4m 35s	remaining: 18m 22s


3000:	learn: 7.5943219	test: 7.7731872	best: 7.7731863 (2997)	total: 6m 48s	remaining: 15m 52s


...

3000:	learn: 7.7038012	test: 7.7739575	best: 7.7739575 (3000)	total: 4m 34s	remaining: 10m 39s


4000:	learn: 7.6351308	test: 7.7685685	best: 7.7685685 (4000)	total: 6m 2s	remaining: 9m 4s


5000:	learn: 7.5692411	test: 7.7650663	best: 7.7650562 (4999)	total: 7m 31s	remaining: 7m 31s


6000:	learn: 7.5046803	test: 7.7627686	best: 7.7627521 (5996)	total: 9m	remaining: 6m


7000:	learn: 7.4419943	test: 7.7613118	best: 7.7612576 (6976)	total: 10m 29s	remaining: 4m 29s


In [ ]:
study.best_params

# Log

 7.762295653542073 no noise ver1
 
 7.761018088737071 no noise ver2 (narrow space)
 
 7.757631368911015 no noise ver3 (narrow space)